In [37]:
import math
import numpy as np

#Resistor design for fixed voltage
#Uout = 1.25V * (1 + R2/R1) , voltage devider, acc. to datasheet
# -> R2/R1 = Uout/1.25V - 1
Uout = 28
R2_R1 = Uout/1.25 - 1
R1 = 2200
R2 = R2_R1*R1
print(R2)

#Resistor design for variable voltage
#controllable by control voltage Uc
#e.g. Uc = 0.... 3.3V
#                   o Uout
#                   |
#                   R2
#                   |
#   Uc o----- Rc ---+----  1.25V (to Pin 5)
#                   |
#                   R1
#                   |
#                   _  0V
#
#
#   I_R1 = 1.25V / R1 = (Uout-1.25V)/R2 + (Uc-1.25V)/Rc
#  -> (Uout-1.25V)*R1/R2  + (Uc-1.25V)*R1/Rc = 1.25V
#   Uc = 0V -> Uout = Uout_max
#   Uc = Uc_mac (=3.3V) -> Uout = Uout_min
#   
#   ->    (Uout_max-1.25V)*R1_R2   +  (0V-1.25V)*R1_Rc = 1.25V
#   ->    (Uout_min-1.25V)*R1_R2   +  (Uc_max-1.25V)*R1_Rc = 1.25V
Uout_max = 28
Uout_min = 5
Uc_max = 3.3
R_ratios_matrix = np.array([[Uout_max-1.25, -1.25],[Uout_min-1.25, Uc_max-1.25]])
R_ratios_rightSide = np.array([1.25, 1.25])
[R1_R2, R1_Rc] = np.linalg.solve(R_ratios_matrix, R_ratios_rightSide)
R1 = 2200
R2 = R1/R1_R2
Rc = R1/R1_Rc
print(R2)
print(Rc)



47080.0
31746.666666666668
4554.95652173913


In [38]:
import math

Uout = 28
Uf = 0.7 #Diode forward voltage
Uin_min = 2.8
Usat = 1.0

Iout_max = 0.1

fs = 40000
Ts = 1/fs

ton_toff = (Uout + Uf - Uin_min)/(Uin_min-Usat)
t_off = Ts / (ton_toff+1)
t_on = Ts-t_off

#print(ton_toff)
print(t_on)
#print(t_off)
print(Ts)
Ct = 4e-5*t_on
print(Ct)

Isw_pk = 2*Iout_max * (ton_toff+1)
print(Isw_pk)

R_sc = 0.3/Isw_pk             #300mV voltage drop
print(R_sc)

L_min = (Uin_min-Usat)/Isw_pk * t_on
print(L_min)

2.3375451263537906e-05
2.5e-05
9.350180505415162e-10
3.077777777777778
0.09747292418772563
1.3670841533188231e-05
